# Clade Labelling

Development workbook for clade-labelling functions. 

### Input
 - a tree file, with some reference names embedded in there.
 - list of reference names

### Output
A dataframe with a list of tipnames, and their clade labels. 

It's possible to not have a clade label if that tipname is too far away from even the nearest reference to be meaningfully determined to "belong" to that reference.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection, LineCollection
import matplotlib.path as mpath
import matplotlib.lines as mlines
from matplotlib import gridspec

import numpy as np
import pandas as pd
import os
os.chdir("/users/dten0001/Google Drive/baltic3/examples")
import sys
from functools import reduce
import subprocess
import time

# Set random state
my_randomstate = np.random.RandomState(12345)

import xio
import my_utils as xu
import biophylo_utils as bpu
import baltic3 as bt
import baltic3_utils as btu

from Bio import Phylo


def tip_to_tip_distance(my_tree, tip1, tip2):
    """Computes the tip-to-mrca-to-tip distance between tip1 and tip2.

    PARAMS
    ------
    my_tree: biopython tree object.
    tip1, tip2: biopython Clade objects

    RETURNS
    -------
    tt_dist: float. tip-to-mrca-to-tip distance between tip1 and tip2.
    """

    mrca = my_tree.common_ancestor(tip1, tip2)
    t1_mrca_dist = my_tree.distance(tip1, mrca)
    t2_mrca_dist = my_tree.distance(tip2, mrca)

    tt_dist = t1_mrca_dist + t2_mrca_dist
    return tt_dist


def get_tt_dist_from_tree(my_tree, ref_names_ls, verbose=True):
    
    t0 = time.time()
    
    # Get all tipnames
    names_ls = [x.name for x in tree.get_terminals()]
    
    # Check
    for ref_nm in ref_names_ls:
        if ref_nm not in names_ls:
            print("WARNING: %s not found in input tree!" % ref_nm)

    # Get the non-reference names
    non_ref_names_ls = list(set.difference(set(names_ls), set(ref_names_ls)))
    
    if verbose:
        print("No. of reference tip names = %s" % len(ref_names_ls))
        print("No. of non-reference tip names = %s" % len(non_ref_names_ls))

    # Compute all possible distances between all tips and all references
    contents = []
    for nm in non_ref_names_ls:
        line = [nm]
        for ref_nm in ref_names_ls:
            tt_dist = tip_to_tip_distance(my_tree, nm, ref_nm)
            line.append(tt_dist)
            
        contents.append(line)

    col_names = ["tip_name"]
    for ref_nm in ref_names_ls:
        col_names.append("dist_to_"+ref_nm)

    df = pd.DataFrame(data=contents, 
                      columns=col_names)
    
    if verbose:
        print("Done in %.2fs" % (time.time() - t0))
        
    return df


def get_clade_labels(df):
    """Given a dataframe that's the output from get_tt_dist_from_tree(),
    Compute the minimum distance as an additional col, and the
    corresponding nearest reference.
    """
    colnames = df.columns
    #df['min_dist'] = df.loc[:, colnames].min(axis=1)
    print("WIP")
    

In [4]:
tree = Phylo.read("H3_test_tree.nex", 'nexus')
tree.root_at_midpoint()
tree.ladderize()

names_ls = [x.name for x in tree.get_terminals()]
ref_names_ls = []
for nm in names_ls:
    if nm.split("|")[0] == "'A/H3N2":
        ref_names_ls.append(nm)

df = get_tt_dist_from_tree(tree, ref_names_ls)

No. of reference tip names = 7
No. of non-reference tip names = 593
Done in 7.70s


In [5]:
df_cols = list(df.columns)[1:]
df["min_col"] = df[df_cols].idxmin(axis=1)
df["min_dist"] = df.loc[:, df_cols].min(axis=1)

In [6]:
df

tip_name  \
0    'H3N2|A/Tasmania/3/2016|70811-HA|H3|TAS|2016-0...   
1    'H3N2|A/Sydney/224/2015|70795-HA|H3|NSW/StMary...   
2    'H3N2|A/Victoria/21/2015|70701-HA|H3|VIC/Kew|2...   
3    'H3N2|A/Sydney/1005/2014|70521-HA|H3|NSW/Yagoo...   
4    'H3N2|A/SouthAustralia/108/2016|70908-HA|H3|SA...   
5    'H3N2|A/Sydney/53/2015|70681-HA|H3|NSW/Sydney|...   
6    'H3N2|A/Newcastle/190/2016|70925-HA|H3|NSW|201...   
7    'H3N2|A/Brisbane/299/2016|70957-HA|H3|QLD/More...   
8    'H3N2|A/Canberra/127/2013|70480-HA|H3|ACT/Canb...   
9    'H3N2|A/Sydney/233/2012|70434-HA|H3|NSW/Hallid...   
10   'H3N2|A/Victoria/7/2015|70631-HA|H3|VIC/Warrna...   
11   'H3N2|A/Victoria/3036/2015|70745-HA|H3|VIC/For...   
12   'H3N2|A/Townsville/8/2015|70584-HA|H3|QLD/Cair...   
13   'H3N2|A/Sydney/1011/2015|70710-HA|H3|NSW/Yagoo...   
14   'H3N2|A/SouthAustralia/135/2015|70764-HA|H3|SA...   
15   'H3N2|A/SouthAustralia/1/2012|70381-HA|H3|SA|2...   
16   'H3N2|A/Sydney/200/2012|70393-HA|H3|NSW/Sydney...   
17   'H3N2|A/Canberra/93/2016|70938-HA|H3|ACT/Wannt...   
18   'H3N2|A/SouthAustralia/1018/2015|70712-HA|H3|S...   
19   'H3N2|A/Perth/26/2015|70610-HA|H3|WA/Hillarys|...   
20   'H3N2|A/Victoria/5075/2014|70547-HA|H3|VIC/Wan...   
21   'H3N2|A/Newcastle/12/2015|70608-HA|H3|NSW/Magi...   
22   'H3N2|A/Victoria/852/2015|70677-HA|H3|VIC/Donc...   
23   'H3N2|A/SouthAustralia/1006/2015|70750-HA|H3|S...   
24   'H3N2|A/Brisbane/158/2016|70869-HA|H3|QLD/Bris...   
25   'H3N2|A/Brisbane/1/2017|70973-HA|H3|QLD/Sunshi...   
26   'H3N2|A/Victoria/504/2013|70464-HA|H3|VIC/Nobl...   
27   'H3N2|A/Brisbane/298/2016|70956-HA|H3|QLD/Wond...   
28   'H3N2|A/Brisbane/1/2013|70459-HA|H3|QLD/Brisba...   
29   'H3N2|A/Brisbane/80/2015|70606-HA|H3|QLD/Rockh...   
..                                                 ...   
563  'H3N2|A/Townsville/51/2016|70971-HA|H3|QLD/Mac...   
564  'H3N2|A/SouthAustralia/133/2015|70760-HA|H3|SA...   
565  'H3N2|A/Brisbane/1008/2016|70895-HA|H3|QLD/Too...   
566  'H3N2|A/Sydney/1042/2015|70771-HA|H3|NSW/Trewi...   
567  'H3N2|A/Sydney/1015/2013|70476-HA|H3|NSW/Cooge...   
568  'H3N2|A/Sydney/57/2015|70672-HA|H3|NSW/Sydney|...   
569  'H3N2|A/SouthAustralia/36/2012|70396-HA|H3|SA|...   
570  'H3N2|A/Brisbane/102/2015|70622-HA|H3|QLD/Bris...   
571  'H3N2|A/Canberra/3/2014|70491-HA|H3|ACT/Canber...   
572  'H3N2|A/Victoria/519/2015|70650-HA|H3|VIC/Nott...   
573  'H3N2|A/Canberra/13/2015|70598-HA|H3|ACT/Canbe...   
574  'H3N2|A/Sydney/1013/2015|70738-HA|H3|NSW/WestB...   
575  'H3N2|A/Victoria/48/2015|70723-HA|H3|VIC/Donca...   
576  'H3N2|A/Victoria/265/2014|70524-HA|H3|VIC/Glen...   
577  'H3N2|A/Sydney/140/2016|70856-HA|H3|NSW/Kelso|...   
578  'H3N2|A/Sydney/1019/2015|70739-HA|H3|NSW/PittT...   
579  'H3N2|A/Sydney/30/2014|70495-HA|H3|NSW/GreenVa...   
580  'H3N2|A/Victoria/3054/2015|70753-HA|H3|VIC/Lil...   
581  'H3N2|A/SouthAustralia/115/2016|70911-HA|H3|SA...   
582  'H3N2|A/Victoria/78/2012|70406-HA|H3|VIC|2012-...   
583  'H3N2|A/Victoria/123/2016|70934-HA|H3|VIC/Kew|...   
584  'H3N2|A/Sydney/129/2016|70839-HA|H3|NSW/Glenmo...   
585  'H3N2|A/Victoria/115/2014|70550-HA|H3|VIC/Hawt...   
586  'H3N2|A/Victoria/5006/2014|70534-HA|H3|VIC/Ash...   
587  'H3N2|A/Victoria/546/2016|70864-HA|H3|VIC/Nobl...   
588  'H3N2|A/Newcastle/187/2016|70924-HA|H3|NSW/Bel...   
589  'H3N2|A/Brisbane/198/2013|70481-HA|H3|QLD/Bris...   
590  'H3N2|A/SouthAustralia/79/2015|70641-HA|H3|SA/...   
591  'H3N2|A/Victoria/5065/2014|70542-HA|H3|VIC/Haw...   
592  'H3N2|A/Sydney/8/2014|70485-HA|H3|NSW/Punchbow...   

     dist_to_'A/H3N2|A/Victoria/361/2011|Australia|2011-10-24'  \
0                                             0.020387           
1                                             0.019144           
2                                             0.017905           
3                                             0.013582           
4                                             0.020348           
5                                             0.01790

In [ ]:
df_dict = {}
for seg in seg_ls:
    print(seg)
    
    
# Rename columns
df_dict2 = {}
for seg in seg_ls:
    d_temp = df_dict[seg][["v_name", "clade_label", "tt_dist_min"]]
    d_temp.columns = ["v_name", "clade_label_"+seg, "tt_dist_min_"+seg]
    df_dict2[seg] = d_temp
    
# Merge the required columns    
df_ls = [df_dict2[seg] for seg in seg_ls]
dz = reduce(lambda left, right: pd.merge(left, right, on="v_name"), df_ls)